# FEM-BEM coupling for a bar antenna

In [ ]:
from ngsolve import *
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve.krylovspace import CG, GMRes
from ngsolve.webgui import Draw
from ngbem import *

In [ ]:
sp = Sphere( (0,0,0), 3)
sp.faces.name="outer"
bar = Cylinder ( (-0.75, 0, 0), X, r=0.1, h=1.5)
bar.faces.name="antenna"
port = Cylinder ( (-0.05, 0, 0), X, r=0.1, h=0.1)
port.faces.name="port"
antenna = bar-port

air = sp-antenna-port

In [ ]:
Draw (air, clipping=True);

In [ ]:
mesh = Mesh(OCCGeometry(air).GenerateMesh(maxh=1)).Curve(2)

In [ ]:
Draw (mesh, clipping=True);

In [ ]:
f = 300e6
omega = 2*pi*f
eps0 = 1.248e-12
mu0 = 1.257e-6

kappa = omega * sqrt(eps0*mu0)
print ("kappa=", kappa)

In [ ]:
fes = HCurl(mesh, order=1, complex=True, dirichlet="antenna")
print ("ndof =", fes.ndof)

## Solve with first order absorbing boundary conditions

In [ ]:
u,v = fes.TnT()
bfa = BilinearForm(1/kappa*curl(u)*curl(v)*dx-kappa* u*v*dx \
                   -1j*u.Trace()*v.Trace()*ds("outer"))
lff = LinearForm( CF((1,0,0)) * v.Trace()*ds("port"))

with TaskManager():
    bfa.Assemble()
    lff.Assemble()
    inv = bfa.mat.Inverse(freedofs=fes.FreeDofs(), inverse="sparsecholesky")

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = inv * lff.vec
Draw(gfu[1], mesh, clipping={"function":True, "z" : -1}, animate_complex=True, min=-0.01, max=0.01, order=2);

In [ ]:
fesHDiv = HDivSurface(mesh, order=1, complex=True, definedon=mesh.Boundaries("outer"))
f2 = LinearForm(fesHDiv).Assemble()

In [ ]:
with TaskManager():
    M = BilinearForm(fes.TrialFunction().Trace()*fesHDiv.TestFunction().Trace()*ds("outer", bonus_intorder=3)).Assemble()
    V = MaxwellSingleLayerPotentialOperator(fesHDiv, kappa, 
                                            intorder=8, leafsize=40, eta=3., eps=1e-4, method="aca")
    K = MaxwellDoubleLayerPotentialOperator(fes, fesHDiv, kappa, trial_definedon=mesh.Boundaries("outer"),
                                            intorder=8, leafsize=40, eta=3., eps=1e-6, method="aca")
    D = MaxwellSingleLayerPotentialOperatorCurl(fes, kappa, definedon=mesh.Boundaries("outer"),
                                            intorder=8, leafsize=40, eta=3., eps=1e-4, method="aca")    

In [ ]:
bfa2 = BilinearForm(1/kappa*curl(u)*curl(v)*dx-kappa*u*v*dx).Assemble()

blockmat = BlockMatrix ([[bfa2.mat-D.mat, (-0.5*M.mat+K.mat).T], [(-0.5*M.mat+K.mat), -V.mat]])
blockvec = BlockVector([lff.vec, f2.vec])

masshdiv = BilinearForm( fesHDiv.TrialFunction().Trace()*fesHDiv.TestFunction().Trace()*ds).Assemble()
pre = BlockMatrix ([[inv, None], [None, 1j*masshdiv.mat.Inverse(freedofs=fesHDiv.FreeDofs())] ])


In [ ]:
sol = GMRes(A=blockmat, b=blockvec, pre=pre, tol=1e-8, maxsteps=200, printrates=True)

In [ ]:
gfu = GridFunction(fes)
gfu.vec[:] = sol[0]
Draw(gfu[1], mesh, clipping={"function":True, "z" : -1}, animate_complex=True, min=-1e-2, max=1e-2, order=2);